# evaluation

### import package

In [1]:
import os
import torch
import numpy as np
import math
import glob
from os import listdir
from os import walk
from torch import nn
from tqdm import tqdm 
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization

import PIL.Image as Image
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torchvision.models as models

### check gpu

In [2]:
data_path = '../../dataset/bird_datasets/train'
classes_path = '../../dataset/bird_datasets/classes.txt'
training_labels_path = '../../dataset/bird_datasets/training_labels.txt'

BATCH_SIZE = 32
WORKERS = 32

print('torch version:' + torch.__version__)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Available GPUs: ', end='')
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i), end=' ')
else:
    device = torch.device('cpu')
    print('CUDA is not available.')

torch version:1.9.0+cu102
Available GPUs: GeForce RTX 2080 Ti GeForce GTX 1080 Ti 

## read classes, labels txt

In [3]:
class_to_idx = {}
with open(classes_path) as f:
    for line in f.readlines():
        label_num =  line.split(".")[0] 
        label_str =  line.split(".")[1][:-1]
        class_to_idx[int(label_num) - 1] = label_str
# print(class_to_idx)

data_list = []
with open(training_labels_path) as f:
    for line in f.readlines():
        file_name =  line.split(" ")[0]
        label_num =  int(line.split(" ")[1].split(".")[0]) -1
        label_str =  line.split(" ")[1].split(".")[1][:-1]
        data_list.append([file_name, label_num, label_str])

train_data_list = data_list[:int(len(data_list) * 0.8)]
val_data_list = data_list[int(len(data_list) * 0.8):int(len(data_list) * 0.9)]
test_data_list = data_list[int(len(data_list) * 0.9):]

# print("all data : ", len(data_list))
# print("train data : ", len(train_data_list))
# print("val data : ", len(val_data_list))
# print("test data : ", len(test_data_list))
# print(train_data_list[:10])

### Data augmentation

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def get_trnsform():
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(1., 1.)),
        transforms.ToTensor(),
        normalize
    ])
    return transform
trans = get_trnsform()

### Dataset
#### Define dataset, and dataloader

In [5]:
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset(dir, data_list):
    images = []
    for img_name, idx, labels in data_list:
        item = (img_name, int(idx))
        images.append(item)
    return images

class BirdImageLoader(Dataset):
    def __init__(self, root, data_list, class_to_idx, transform=None, target_transform=None):
        imgs = make_dataset(root, data_list)

        self.root = root
        self.imgs = imgs
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
dataset_test = BirdImageLoader(data_path, test_data_list, class_to_idx, transform=trans)

test_loader = DataLoader(
    dataset_test,
    num_workers=WORKERS,
    batch_size=BATCH_SIZE,
    shuffle=False
)
dataset_test.__len__()

/home/me/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 20, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


300

### test model

In [7]:
crossEntropyLoss_fn = torch.nn.CrossEntropyLoss()
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def pass_epoch(model, loader, mode = 'Train'):
    loss = 0
    loss_acc_top1 = 0
    loss_acc_top5 = 0
    
    for i_batch, image_batch in tqdm(enumerate(loader)):
        x, y = image_batch[0].to(device), image_batch[1].to(device)
        if mode == 'Train':
            model.train()
        elif mode == 'Eval':
            model.eval()
        else:
            print('error model mode!')
        y_pred = model(x)

        loss_batch = crossEntropyLoss_fn(y_pred, y)
        loss_batch_acc_top = accuracy(y_pred, y, topk=(1, 5))

        if mode == 'Train':
            model_optimizer.zero_grad()
            loss_batch.backward()
            model_optimizer.step()
        
        loss += loss_batch.detach().cpu()
        loss_acc_top1 += loss_batch_acc_top[0]
        loss_acc_top5 += loss_batch_acc_top[1]
        
    loss /= (i_batch + 1)
    loss_acc_top1 /= (i_batch + 1)
    loss_acc_top5 /= (i_batch + 1)
    
    return loss, loss_acc_top1, loss_acc_top5

In [8]:
def test_model(path, idx = 99):
    with torch.no_grad():
        model = torch.load('{}/checkpoint.pth.tar'.format(path))
#         model = models.resnet50(pretrained=True).to(device)
        model.load_state_dict(torch.load('{}/checkpoint_{}.pth.tar'.format(path, idx))['state_dict'])
        val_loss, val_acc_top1, val_acc_top5 = pass_epoch(model,test_loader, 'Eval') 
    print("val_loss ", val_loss.to('cpu').numpy(), "val_acc_top1", val_acc_top1.to('cpu').numpy(), "val_acc_top5 ", val_acc_top5.to('cpu').numpy())
    torch.cuda.empty_cache()

In [12]:
test_model('model/model_bird_res50_LR5', '0015')

10it [00:01,  7.55it/s]

tensor(2.8443) tensor([46.8750], device='cuda:0') tensor([78.0208], device='cuda:0')


In [85]:
test_model('model/model_bird_resnet50', '0015')

10it [00:01,  7.26it/s]

tensor(3.5304) tensor([43.3333], device='cuda:0') tensor([71.4583], device='cuda:0')


In [87]:
test_model('model/model_bird_res50_SGD', '0050')

10it [00:00, 10.65it/s]

tensor(2.7048) tensor([49.7917], device='cuda:0') tensor([73.5417], device='cuda:0')


In [88]:
test_model('model/model_bird_res50_SGD', '0060')

10it [00:01,  7.13it/s]

tensor(2.6031) tensor([45.5208], device='cuda:0') tensor([77.8125], device='cuda:0')


In [90]:
test_model('model/model_bird_resnet50_all_aug', '0005')

10it [00:00, 10.16it/s]

tensor(2.1976) tensor([48.6458], device='cuda:0') tensor([77.3958], device='cuda:0')


In [96]:
test_model('model/model_bird_resnet50_all_aug', '0019')

10it [00:01,  7.40it/s]

tensor(2.3349) tensor([46.2500], device='cuda:0') tensor([76.9792], device='cuda:0')


In [13]:
test_model('model/model_bird_resnet50_all_aug', '0095')

10it [00:00, 10.27it/s]

tensor(2.8569) tensor([55.6250], device='cuda:0') tensor([74.7917], device='cuda:0')


In [10]:
test_model('model/model_bird_resnet50_all_aug_RL', '0022')

10it [00:01,  8.89it/s]

tensor(1.7659) tensor([55.9375], device='cuda:0') tensor([83.8542], device='cuda:0')


In [14]:
test_model('model/model_bird_resnet50_all_aug_RL', '0027')

10it [00:01,  6.70it/s]

tensor(1.7799) tensor([55.9375], device='cuda:0') tensor([82.7083], device='cuda:0')


In [21]:
test_model('model/model_bird_vit', '0028')

10it [00:01,  5.61it/s]

tensor(1.7091) tensor([66.3542], device='cuda:0') tensor([92.2917], device='cuda:0')


In [22]:
test_model('model/model_bird_vit_MultiStepLR', '0050')

10it [00:03,  2.58it/s]

tensor(1.0902) tensor([76.8750], device='cuda:0') tensor([96.5625], device='cuda:0')


In [34]:
test_model('model/model_bird_vit_ReduceLROnPlateau', '0046')

10it [00:03,  2.88it/s]

val_loss  0.8201518 val_acc_top1 [78.75] val_acc_top5  [96.25]


In [40]:
test_model('model/model_bird_vit_SGD', '0047')

10it [00:01,  5.33it/s]

val_loss  0.63634455 val_acc_top1 [80.625] val_acc_top5  [96.875]


In [43]:
test_model('model/model_bird_vit_CrossEntropyLS', '0047')

10it [00:01,  5.46it/s]

val_loss  0.7801565 val_acc_top1 [81.875] val_acc_top5  [97.8125]


In [21]:
test_model('model/model_bird_vit_AllData', '0047')

10it [00:01,  5.52it/s]

val_loss  0.18112554 val_acc_top1 [98.75] val_acc_top5  [100.]


In [15]:
test_model('model/model_bird_vit_CrossEntropyLS', '0074')

10it [00:03,  2.86it/s]

val_loss  0.80037576 val_acc_top1 [81.5625] val_acc_top5  [97.8125]


In [12]:
test_model('model/model_bird_vit_vic', '0031')

10it [00:03,  2.90it/s]

val_loss  0.7540843 val_acc_top1 [82.5] val_acc_top5  [97.5]


In [16]:
test_model('model/model_bird_vit_vic', '0035')

10it [00:03,  2.83it/s]

val_loss  0.7697259 val_acc_top1 [81.25] val_acc_top5  [97.1875]
